In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import random
import os
import matplotlib.pyplot as plt
from sklearn.metrics import auc, roc_curve, confusion_matrix, classification_report
import pandas as pd
import numpy as np
import subprocess
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score,balanced_accuracy_score
from sklearn.model_selection import StratifiedKFold,RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
data = pd.read_excel('/Users/rachael/Downloads/train_df_labelled copy.xlsx')
#the data verified by the third person will be used as the train set finally
file =  data[['content','sentiment']]
file['sentiment'].value_counts()
label_sentiment= file['sentiment'].value_counts()
label_proportions_sentiment= file['sentiment'].value_counts(normalize=True)
print("Counts of each label_sentiment:")
print(label_sentiment)
print("\nProportions of each label_sentiment:")
print(label_proportions_sentiment)

Counts of each label_sentiment:
sentiment
Negative    262
Neither      99
Positive     48
Name: count, dtype: int64

Proportions of each label_sentiment:
sentiment
Negative    0.640587
Neither     0.242054
Positive    0.117359
Name: proportion, dtype: float64


In [3]:
import re
def remove_urls(text):
    # 定义URL的正则表达式
    url_pattern = r'https?://\S+|www\.\S+'
    # 使用re.sub()函数替换URL为空字符串
    no_url_text = re.sub(url_pattern, '', text)
    return no_url_text

def clean_text(text):
    # Remove anything that is not a letter or space
    clean_text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Optional: Convert text to lower case
    clean_text = clean_text.lower()
    return clean_text

#remove stopwords
def remove_stopwords(text, stopwords):
    words = re.findall(r'\b\w+\b', text.lower())
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

dutch_stopwords = ['de', 'en', 'van', 'ik', 'te', 'dat', 
                  'die', 'in', 'een', 'hij', 'het', 'niet', 
                  'zijn', 'is', 'was', 'op', 'aan', 'met', 'als', 
                  'voor', 'had', 'er', 'maar', 'om', 'hem', 'dan', 
                  'zou', 'of', 'wat', 'mijn', 'men', 'dit', 'zo', 
                  'door', 'over', 'ze', 'zich', 'bij', 'ook', 'tot', 
                  'je', 'mij', 'uit', 'der', 'daar', 'haar', 'naar', 
                  'heb', 'hoe', 'heeft', 'hebben', 'deze', 'u', 'want', 
                  'nog', 'zal', 'me', 'zij', 'nu', 'ge', 'geen', 'omdat', 
                  'iets', 'worden', 'toch', 'al', 'waren', 'veel', 'meer', 
                  'doen', 'toen', 'moet', 'ben', 'zonder', 'kan', 'hun', 
                  'dus', 'alles', 'onder', 'ja', 'eens', 'hier', 'wie', 
                  'werd', 'altijd', 'doch', 'wordt', 'wezen', 'kunnen', 
                  'ons', 'zelf', 'tegen', 'na', 'reeds', 'wil', 'kon', 
                  'niets', 'uw', 'iemand', 'geweest', 'andere']

In [4]:
file['content'] = file['content'].apply(remove_urls) 
file['content'] = file['content'].apply(clean_text) 
file['content'] = file['content'].apply(lambda x: remove_stopwords(x, dutch_stopwords))
file['sentiment'] = file['sentiment'].replace({'Positive':0,'Negative': 1, 'Neither': 2})
y = file['sentiment']
X = file['content']

/var/folders/vq/5p2k6x290wzfc5dzz5xm78bm0000gp/T/ipykernel_5363/4109358769.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file['content'] = file['content'].apply(remove_urls)
/var/folders/vq/5p2k6x290wzfc5dzz5xm78bm0000gp/T/ipykernel_5363/4109358769.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file['content'] = file['content'].apply(clean_text)
/var/folders/vq/5p2k6x290wzfc5dzz5xm78bm0000gp/T/ipykernel_5363/4109358769.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [5]:
param_dist= {
    'n_estimators': range(10, 101),
    'criterion': ['gini', 'entropy'],
    'max_depth': range(2, 51),
    'min_samples_split': range(2, 11),
    'min_samples_leaf': range(1, 11),
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}

best_score = 0
best_params = {}

test_scores = []
best_params_list = []
tfidf_vectorizer = TfidfVectorizer()
f1_scores = []

In [6]:
# Stratified K-fold for maintaining label distribution,shuffle=True确保每次迭代的数据划分不同
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Loop over each fold
for train_index, test_index in outer_cv.split(X, y):
    trainX, testX = X.iloc[train_index], X.iloc[test_index]
    trainy, testy = y.iloc[train_index], y.iloc[test_index]
   
    # Apply TF-IDF Vectorization
    X_train_tfidf = tfidf_vectorizer.fit_transform(trainX)
    X_test_tfidf = tfidf_vectorizer.transform(testX)
    
    rf = RandomForestClassifier(random_state=42,class_weight='balanced')
    randomized_search = RandomizedSearchCV(
        estimator=rf,
        param_distributions=param_dist,
        n_iter=100,  # Number of parameter settings that are sampled
        scoring='f1_macro',
        n_jobs=-1,  # Use all available cores
        cv=inner_cv,
        random_state=42
    )

    # Fit Randomized Search
    randomized_search.fit(X_train_tfidf, trainy)

    # Get the best model from Randomized Search
    best_model = randomized_search.best_estimator_
    best_params = randomized_search.best_params_
    best_score = randomized_search.best_score_

    # Evaluate on the test set
    test_predictions = best_model.predict(X_test_tfidf)
    test_score = f1_score(testy, test_predictions,average='macro')
    

    test_scores.append(test_score)
    best_params_list.append(best_params)

    print(f"Fold test score: {test_score}")
    print(f"Best parameters: {best_params}")

# Summarize the results
mean_test_score = np.mean(test_scores)
std_test_score = np.std(test_scores)

print(f"Mean test score: {mean_test_score}")
print(f"Standard deviation of test scores: {std_test_score}")
print("Best parameters for each fold:")
for i, params in enumerate(best_params_list):
    print(f"Fold {i + 1}: {params}")

Fold test score: 0.2765911542610571
Best parameters: {'n_estimators': 26, 'min_samples_split': 9, 'min_samples_leaf': 3, 'max_features': None, 'max_depth': 4, 'criterion': 'entropy', 'bootstrap': True}
Fold test score: 0.3313000381242852
Best parameters: {'n_estimators': 24, 'min_samples_split': 10, 'min_samples_leaf': 10, 'max_features': None, 'max_depth': 42, 'criterion': 'gini', 'bootstrap': True}
Fold test score: 0.3612006590729995
Best parameters: {'n_estimators': 84, 'min_samples_split': 10, 'min_samples_leaf': 7, 'max_features': None, 'max_depth': 16, 'criterion': 'entropy', 'bootstrap': True}
Fold test score: 0.4065349143610013
Best parameters: {'n_estimators': 92, 'min_samples_split': 9, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 14, 'criterion': 'gini', 'bootstrap': False}
Fold test score: 0.3614232209737828
Best parameters: {'n_estimators': 95, 'min_samples_split': 6, 'min_samples_leaf': 5, 'max_features': None, 'max_depth': 9, 'criterion': 'entropy', 'boots

In [7]:
from collections import Counter
best_params_counter = Counter(tuple(sorted(params.items())) for params in best_params_list)
most_common_params = dict(best_params_counter.most_common(1)[0][0])

print("Most common best parameters:", most_common_params)

Most common best parameters: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 4, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 9, 'n_estimators': 26}


In [5]:
most_common_params = {'n_estimators': 92, 'min_samples_split': 9, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 14, 'criterion': 'gini', 'bootstrap': False}


In [6]:
file = pd.read_excel('/Users/rachael/Downloads/train_df_labelled copy.xlsx')
file['content'] = file['content'].apply(remove_urls)
file['content'] = file['content'].apply(clean_text)
file['content'] = file['content'].apply(lambda x: remove_stopwords(x, dutch_stopwords))
file['sentiment'] = file['sentiment'].replace({'Positive': 0, 'Negative': 1, 'Neither': 2})
y = file['sentiment']
X = file['content'] 
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X)
final_model = RandomForestClassifier(random_state=42, **most_common_params,class_weight='balanced')
final_model.fit(X_train_tfidf, y)

RandomForestClassifier(bootstrap=False, class_weight='balanced', max_depth=14,
                       min_samples_leaf=2, min_samples_split=9, n_estimators=92,
                       random_state=42)

In [7]:
#apply the model to the test set
test = pd.read_excel('/Users/rachael/Downloads/test_df_labelled.xlsx')
test = test[['content', 'sentiment']]
test['content'] = test['content'].apply(remove_urls)
test['content'] = test['content'].apply(clean_text)
test['content'] = test['content'].apply(lambda x: remove_stopwords(x, dutch_stopwords))
test['sentiment'] = test['sentiment'].replace({'Positive': 0, 'Negative': 1, 'Neither': 2})

# Extract features and labels from test data
y_test = test['sentiment'].astype(int)
X_test = test['content']

# Transform the test data using the same TF-IDF vectorizer
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Make predictions on the test data using the trained model
test_predictions = final_model.predict(X_test_tfidf)
test_score = accuracy_score(y_test, test_predictions)

# Print test results
print(f"Test Accuracy: {test_score}")

Test Accuracy: 0.5135135135135135


In [8]:
test_score = accuracy_score(y_test, test_predictions)
conf_matrix = confusion_matrix(y_test, test_predictions)
class_report = classification_report(y_test, test_predictions)
print(class_report)

              precision    recall  f1-score   support

           0       0.19      0.29      0.23        38
           1       0.64      0.55      0.59       245
           2       0.46      0.51      0.48       124

    accuracy                           0.51       407
   macro avg       0.43      0.45      0.43       407
weighted avg       0.54      0.51      0.52       407

